In [101]:
!pip install torch torchvision pennylane matplotlib seaborn opencv-python numpy scikit-learn

In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pennylane as qml
import numpy as np
import glob, cv2, random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns

device = torch.device(
    'mps' if torch.backends.mps.is_available() else
    'cuda' if torch.cuda.is_available() else
    'cpu'
)
print(f'Using device: {device}')


Using device: mps


In [103]:
class Dataset(object):
    """Cette class est une class abstraite representant un Dataset

    Toute autre class de dataset devrait etre une sous class de celle-ci.
    Et chaque class devrait 'Ecraser' ``__len__``, qui retourne la taille du dataset, et
    ``__getitem__``, qui supporte les index en entier qui va de 0 a len(self) exclusive.
    """
    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

In [104]:
class IRM(Dataset):
    def __init__(self):
        
        tumor = []
        healthy = []
        # cv2 - It reads in BGR format by default
        for f in glob.iglob("../data/brain_tumor_dataset/yes/*.jpg"):
            img = cv2.imread(f)
            img = cv2.resize(img,(128,128)) 
            b, g, r = cv2.split(img)
            img = cv2.merge([r,g,b])
            img = img.reshape((img.shape[2],img.shape[0],img.shape[1]))
            tumor.append(img)

        for f in glob.iglob("../data/brain_tumor_dataset/no/*.jpg"):
            img = cv2.imread(f)
            img = cv2.resize(img,(128,128)) 
            b, g, r = cv2.split(img)
            img = cv2.merge([r,g,b])
            img = img.reshape((img.shape[2],img.shape[0],img.shape[1]))
            healthy.append(img)

        # Nos images
        tumor = np.array(tumor,dtype=np.float32)
        healthy = np.array(healthy,dtype=np.float32)
        
        # Nos titres
        tumor_label = np.ones(tumor.shape[0], dtype=np.float32)
        healthy_label = np.zeros(healthy.shape[0], dtype=np.float32)
        
        # Concatenation des deux
        self.images = np.concatenate((tumor, healthy), axis=0)
        self.labels = np.concatenate((tumor_label, healthy_label))
        
    def __len__(self):
        return self.images.shape[0]
    
    def __getitem__(self, index):
        
        sample = {'image': self.images[index], 'label':self.labels[index]}
        
        return sample
    
    def normalize(self):
        self.images = self.images/255.0

In [105]:
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)
rand_params = np.random.uniform(0, 2*np.pi, size=(1, n_qubits))

@qml.qnode(dev)
def quanv_circuit(patch):
    for i in range(n_qubits):
        qml.RY(np.pi * patch[i], wires=i)
    qml.templates.RandomLayers(rand_params, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


In [106]:
class QuanvLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.out_channels = 4

    def forward(self, x):
        bs, c, h, w = x.size()
        out_h, out_w = h // 2, w // 2
        output = torch.zeros((bs, self.out_channels, out_h, out_w), dtype=torch.float32).to(device)

        for b in range(bs):
            for i in range(0, h, 2):
                for j in range(0, w, 2):
                    patch = x[b, :, i:i+2, j:j+2].reshape(-1).detach().cpu().numpy().astype(np.float32)
                    q_results = np.array(quanv_circuit(patch), dtype=np.float32)
                    output[b, :, i//2, j//2] = torch.from_numpy(q_results).to(device)
        return output


In [107]:
class QuantumCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.quanv = QuanvLayer()
        
        # Classical layers exactly as your original CNN
        self.cnn_model = nn.Sequential(
            nn.Conv2d(4, 8, kernel_size=3),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(8, 16, kernel_size=3),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

        self.fc_model = nn.Sequential(
            nn.Linear(16*14*14, 120),
            nn.Tanh(),
            nn.Linear(120, 84),
            nn.Tanh(),
            nn.Linear(84, 1)
        )

    def forward(self, x):
        x = self.quanv(x)
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        return torch.sigmoid(x)

In [108]:
irm_dataset = IRM()
irm_dataset.normalize()
dataloader = DataLoader(irm_dataset, batch_size=32, shuffle=True)

model = QuantumCNN().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

EPOCHS = 20  # reduce initially to verify functionality

model.train()
for epoch in range(EPOCHS):
    epoch_loss = 0
    for batch in dataloader:
        images = batch['image'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{EPOCHS}] Loss: {epoch_loss/len(dataloader):.4f}')


KeyboardInterrupt: 